In [5]:
import pandas as pd
import geopandas as gpd
import numpy as np
import os
import matplotlib.pyplot as plt
import imageio.v2 as imageio
import osmnx as ox 
from shapely.geometry import Point, LineString

In [6]:
# 1. read bus stops geojson
bus_stops = gpd.read_file('data/raw/tfl/Bus_Stops.geojson')
bus_stops = bus_stops.to_crs(epsg=27700)
bus_stops['STOP_CODE'] = bus_stops['STOP_CODE'].astype(str).str.strip() # CONVERT STOP_CODE TO STR

# 2. load bus stop data
bus_stops1 = pd.read_csv('data/raw/tfl/bus-stops.csv')
bus_stops1['Bus_Stop_Code'] = bus_stops1['Bus_Stop_Code'].astype(str).str.strip() # CONVERT STOP_CODE TO STR

# combine bus_stops and bus_stops1 with stop_code
bus_stops = bus_stops.merge(bus_stops1, left_on='STOP_CODE', right_on='Stop_Code_LBSL', how='outer')

# fill stop_code with Stop_Code_LBSL
bus_stops['STOP_CODE'] = bus_stops['STOP_CODE'].fillna(bus_stops['Stop_Code_LBSL'])
bus_stops['STOP_NAME'] = bus_stops['STOP_NAME'].fillna(bus_stops['Stop_Name'])
bus_stops['OS_EASTING'] = bus_stops['OS_EASTING'].fillna(bus_stops['Location_Easting'])
bus_stops['OS_NORTHING'] = bus_stops['OS_NORTHING'].fillna(bus_stops['Location_Northing'])
bus_stops['geometry'] = [Point(x, y) for x, y in zip(bus_stops['OS_EASTING'], bus_stops['OS_NORTHING'])]

# filter to necessary columns
bus_stops = bus_stops[['STOP_CODE', 'STOP_NAME', 'OS_EASTING', 'OS_NORTHING','geometry']]
bus_stops = gpd.GeoDataFrame(bus_stops, geometry='geometry', crs='epsg:27700')
bus_stops.info()

<class 'geopandas.geodataframe.GeoDataFrame'>
RangeIndex: 21540 entries, 0 to 21539
Data columns (total 5 columns):
 #   Column       Non-Null Count  Dtype   
---  ------       --------------  -----   
 0   STOP_CODE    21540 non-null  object  
 1   STOP_NAME    21517 non-null  object  
 2   OS_EASTING   21540 non-null  float64 
 3   OS_NORTHING  21540 non-null  float64 
 4   geometry     21540 non-null  geometry
dtypes: float64(2), geometry(1), object(2)
memory usage: 841.5+ KB


In [11]:
# import bus data
file = os.path.join('data/raw/tfl/busto_sample.CSV')
bus_entex = pd.read_csv(file, sep=';')

# get geometry for bus_entex from bus_stops
bus_entex= bus_entex.merge(bus_stops[['STOP_CODE', 'geometry']], left_on='STOPCODE', right_on='STOP_CODE', how='left')
bus_entex = bus_entex.drop(columns=['STOP_CODE'])
bus_entex = gpd.GeoDataFrame(bus_entex, geometry='geometry',crs='epsg:27700')
bus_entex.info()

<class 'geopandas.geodataframe.GeoDataFrame'>
RangeIndex: 922 entries, 0 to 921
Data columns (total 15 columns):
 #   Column        Non-Null Count  Dtype   
---  ------        --------------  -----   
 0   YEAR          922 non-null    int64   
 1   DAY_TYPE      922 non-null    object  
 2   TIMEBAND      922 non-null    int64   
 3   ROUTE         922 non-null    int64   
 4   DIRECTION     922 non-null    int64   
 5   STOPCODE      922 non-null    object  
 6   STOPNAME      922 non-null    object  
 7   STOPSEQUENCE  922 non-null    float64 
 8   AllBoardings  922 non-null    float64 
 9   AllAlighted   922 non-null    float64 
 10  load          922 non-null    float64 
 11  Capacity      922 non-null    float64 
 12  Seats         922 non-null    float64 
 13  V/C           922 non-null    float64 
 14  geometry      922 non-null    geometry
dtypes: float64(7), geometry(1), int64(4), object(3)
memory usage: 108.2+ KB


FILTER WITHIN LONDON   

In [12]:
# download the Greater London boundary
greater_london = ox.geocode_to_gdf('Greater London, UK')
greater_london = greater_london.to_crs(epsg=27700)

# subset points that are within the boundary of london
bus_entex = bus_entex[bus_entex.geometry.intersects(greater_london.geometry[0])]
bus_entex.info()

<class 'geopandas.geodataframe.GeoDataFrame'>
Index: 877 entries, 0 to 921
Data columns (total 15 columns):
 #   Column        Non-Null Count  Dtype   
---  ------        --------------  -----   
 0   YEAR          877 non-null    int64   
 1   DAY_TYPE      877 non-null    object  
 2   TIMEBAND      877 non-null    int64   
 3   ROUTE         877 non-null    int64   
 4   DIRECTION     877 non-null    int64   
 5   STOPCODE      877 non-null    object  
 6   STOPNAME      877 non-null    object  
 7   STOPSEQUENCE  877 non-null    float64 
 8   AllBoardings  877 non-null    float64 
 9   AllAlighted   877 non-null    float64 
 10  load          877 non-null    float64 
 11  Capacity      877 non-null    float64 
 12  Seats         877 non-null    float64 
 13  V/C           877 non-null    float64 
 14  geometry      877 non-null    geometry
dtypes: float64(7), geometry(1), int64(4), object(3)
memory usage: 109.6+ KB


FUNCTION TO FILTER BUS DATA BY DATE

In [13]:
# filter for saturday only
bus_entex = bus_entex[bus_entex['DAY_TYPE'] == 'Saturday']
bus_entex.info()

<class 'geopandas.geodataframe.GeoDataFrame'>
Index: 877 entries, 0 to 921
Data columns (total 15 columns):
 #   Column        Non-Null Count  Dtype   
---  ------        --------------  -----   
 0   YEAR          877 non-null    int64   
 1   DAY_TYPE      877 non-null    object  
 2   TIMEBAND      877 non-null    int64   
 3   ROUTE         877 non-null    int64   
 4   DIRECTION     877 non-null    int64   
 5   STOPCODE      877 non-null    object  
 6   STOPNAME      877 non-null    object  
 7   STOPSEQUENCE  877 non-null    float64 
 8   AllBoardings  877 non-null    float64 
 9   AllAlighted   877 non-null    float64 
 10  load          877 non-null    float64 
 11  Capacity      877 non-null    float64 
 12  Seats         877 non-null    float64 
 13  V/C           877 non-null    float64 
 14  geometry      877 non-null    geometry
dtypes: float64(7), geometry(1), int64(4), object(3)
memory usage: 109.6+ KB


Split by time band

In [18]:
# pivot by time bands
bus_entex_pivot = bus_entex.pivot_table(index=['STOPCODE','STOPNAME','ROUTE','geometry'], columns='TIMEBAND', values=['AllAlighted'], aggfunc='sum')
bus_entex_pivot.columns = bus_entex_pivot.columns.droplevel()
bus_entex_pivot = bus_entex_pivot.reset_index()
# add total columnS that are numerical
bus_entex_pivot['Total'] = bus_entex_pivot[bus_entex_pivot.columns[4:]].sum(axis=1)
bus_entex_pivot.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 874 entries, 0 to 873
Data columns (total 6 columns):
 #   Column    Non-Null Count  Dtype   
---  ------    --------------  -----   
 0   STOPCODE  874 non-null    object  
 1   STOPNAME  874 non-null    object  
 2   ROUTE     874 non-null    int64   
 3   geometry  874 non-null    geometry
 4   8         874 non-null    float64 
 5   Total     874 non-null    float64 
dtypes: float64(2), geometry(1), int64(1), object(2)
memory usage: 41.1+ KB


In [19]:
# rename columns based on timeband, use dict

timeband_dict = {
    8: 'Early',
    1: 'Early',
    2: 'AM Peak',
    3: 'AM Peak',
    4: 'Midday',
    5: 'PM Peak',
    6: 'Evening',
    7: 'Late',
}
bus_entex_pivot = bus_entex_pivot.rename(columns=timeband_dict)

In [22]:
# geodataframe
bus_entex_pivot = gpd.GeoDataFrame(bus_entex_pivot, geometry='geometry',crs='epsg:27700')
bus_entex_pivot.head()

TIMEBAND,STOPCODE,STOPNAME,ROUTE,geometry,Early,Total
0,1011,CLAPTON POND,106,POINT (534887.000 186004.000),37.92242,37.92242
1,10242,FAIRLAWN AVENUE,102,POINT (527295.000 189380.000),60.84237,60.84237
2,1047,THE OLD VIC,1,POINT (531368.000 179740.000),11.31737,11.31737
3,1048,THE OLD VIC,1,POINT (531345.000 179734.000),58.79182,58.79182
4,10813,WHITGIFT CENTRE,109,POINT (532444.000 165831.000),1.05100,1.05100


EXPORT

In [23]:
# export to geojson AGGREGATED BY STOP AND ROUTE
bus_entex_pivot.to_file('data/cleaned/flow_bus_byStopRoutes_.gpkg', driver='GPKG')

In [24]:
# sum flow_bus by stoap_id, aggregate all numerical columns
flow_bus_byStop = bus_entex_pivot.groupby(['STOPCODE','STOPNAME']).agg({
    'Early': 'sum',
    # 'AM Peak': 'sum',
    # 'Midday': 'sum',
    # 'PM Peak': 'sum',
    # 'Evening': 'sum',
    # 'Late': 'sum',
    'Total': 'sum',
    'geometry': 'first'
}).reset_index()
flow_bus_byStop = gpd.GeoDataFrame(flow_bus_byStop, geometry='geometry', crs='EPSG:27700')
flow_bus_byStop.info()

<class 'geopandas.geodataframe.GeoDataFrame'>
RangeIndex: 868 entries, 0 to 867
Data columns (total 5 columns):
 #   Column    Non-Null Count  Dtype   
---  ------    --------------  -----   
 0   STOPCODE  868 non-null    object  
 1   STOPNAME  868 non-null    object  
 2   Early     868 non-null    float64 
 3   Total     868 non-null    float64 
 4   geometry  868 non-null    geometry
dtypes: float64(2), geometry(1), object(2)
memory usage: 34.0+ KB


In [25]:
# export to geojson AGGREGATED BY STOP
flow_bus_byStop.to_file('data/cleaned/flow_bus_byStop_.gpkg', driver='GPKG')